In [14]:
import regex as re
import pandas as pd
import os
import random
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='your module name')

In [15]:
data = pd.read_csv('smalldata_washed.csv')
rumor = data['rumor'].to_list()
reverse = data['reverse'].to_list()
rumor_class = len(rumor)*[1]
reverse_class = len(reverse)*[0]
data = rumor + reverse
data_class = rumor_class + reverse_class


In [16]:
#后续添加数据要放在raw目录下，使用.xlsx格式将后续标注的数据加入data
raw_data_list = os.listdir('raw')
data_list = []
good_name = re.compile(r'^(?!(\~\$)).*(.xlsx)')
for i in raw_data_list:
    if good_name.match(i):
    #    data_list.append(i)
        temp = pd.read_excel('raw/'+i)
        temp.fillna('',inplace=True)
        temp_rumor = [x.strip() for x in  temp['谣言'].to_list()  if x.strip()!='']
        temp_reverse = [x.strip() for x in  temp['真相'].to_list()  if x.strip()!='']
        temp_rumor_class = len(temp_rumor)*[1]
        temp_reverse_class = len(temp_reverse)*[0]
        temp_data = temp_rumor + temp_reverse
        temp_data_class = temp_rumor_class + temp_reverse_class
        for i in range( len(temp_data)):
            temp_data[i]=re.sub(r'[\,\'\ ]|(\n)','',temp_data[i])
    data = data + temp_data
    data_class = data_class + temp_data_class
with open('data.txt','w') as f:
    for i in range(len(data)):
        f.write(data[i]+'\t'+str(data_class[i])+'\n')

    


In [17]:
data_list =list( zip(data, data_class))
random.shuffle(data_list)
data, data_class = zip(*data_list)
index = ['rumor{}{}'.format(i, '!' * j) for (i, j) in zip(range(len(data)), data_class)]
df = pd .DataFrame(data=zip(data_class,data), columns=["rumor","text"], index=index)

In [18]:
# jieba分词
import jieba
# 去除停用词
with open ('hit_stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.read().splitlines()
    
for i in range(len(df)):
    words = jieba.cut(df["text"].iloc[i],cut_all=False)
    words = [word for word in words if word not in stopwords]
    df["text"].iloc[i] = ' '.join(words) 

/var/folders/f0/6j6j3w814n53skbt2s4d8p5m0000gn/T/ipykernel_43635/1381119319.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"].iloc[i] = ' '.join(words)


#### 使用sklearn计算TF-IDF

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize

In [20]:
tfidf_model = TfidfVectorizer(tokenizer=casual_tokenize,ngram_range=(1,2))
tfidf_docs = tfidf_model.fit_transform(raw_documents=df.text).toarray()#这句话计算了tfidf
tfidf_docs.shape

/Users/stevenyu/opt/anaconda3/envs/nlpiaenv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(1178, 25521)

In [21]:
df.rumor.sum()

365

#### 使用LDA语义分析技术进行聚类

In [22]:
msak = df.rumor.astype(bool).values
spam_centroid = tfidf_docs[msak].mean(axis=0)
ham_centroid = tfidf_docs[~msak].mean(axis=0)
spam_centroid.round(2)



array([0., 0., 0., ..., 0., 0., 0.])

In [23]:
ham_centroid.round(2)

array([0., 0., 0., ..., 0., 0., 0.])

In [24]:
spamminess_score = tfidf_docs.dot(spam_centroid - ham_centroid)
spamminess_score.round(2)

array([-0.  , -0.  , -0.01, ...,  0.  ,  0.  , -0.  ])

In [25]:
from sklearn.preprocessing import MinMaxScaler
df['lda_score'] = MinMaxScaler().fit_transform(spamminess_score.reshape(-1, 1))
df['lda_pred'] = (df.lda_score > .6).astype(int)
df['rumor lda_pred lda_score'.split()].round(2).head(6)

,rumor,lda_pred,lda_score
rumor0,0,1,0.62
rumor1,0,0,0.58
rumor2,0,0,0.42
rumor3,0,0,0.49
rumor4,0,0,0.35
rumor5!,1,1,0.87


In [26]:
(1. - (df.rumor - df.lda_pred).abs().sum() / len(df)).round(4)

0.9669